# Parallel Interaction on JudgeBench with Position Swapping

## Imports

In [1]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import re
from tqdm import tqdm
from collections import Counter

c:\Users\NIBIEBL\Desktop\coding\Bachelorarbeit_Repository\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")

df = pd.DataFrame(JudgeBench_Claude)

df_sampled = df.iloc[:270].sample(n=100, random_state=42).reset_index(drop=True)

df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [3]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42
    }
]

## System Design

In [4]:
agent_1_system_message =f"""
You are a Scientific-Agent with a background in academic research and critical thinking.
Your task is to assess which of Response A or Response B is more factually correct. 
Focus on whether the statements are logically sound, consistent with known facts and science.
Give an explanation for your decision using about 200 words.
Always begin your output with "As a Scientific-Agent I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

agent_2_system_message =f"""
You are a Logical-Agent specializing in evaluating argument consistency, clarity and coherence.
Your task is to analyze Response A and Response B in terms of internal logical structure and factual plausibility.
Look for contradictions, fallacies, and misleading reasoning, even if subtle.
Give an explanation for your decision using about 200 words.
Always begin your output with: "As a Logical-Agent I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

agent_3_system_message =f"""
You are a Domain-Expert-Agent with applied experience in the relevant field.
Your task is to judge which of Response A or Response B reflects real-world knowledge and practical correctness more accurately.
Focus on practical validity, typical use cases, and whether the information would hold in applied scenarios.
Give an explanation for your decision using about 200 words.
Always begin your output with: "As a Domain-Expert-Agent I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

In [5]:
initializer = ConversableAgent(
    "initializer", 
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    )

agent_1 = ConversableAgent(
    "Scientific-Agent",
    llm_config={"config_list": config_list},
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)
agent_2 = ConversableAgent(
    "Logical-Agent",
    llm_config={"config_list": config_list},
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)
agent_3 = ConversableAgent(
    "Domain-Expert",
    llm_config={"config_list": config_list},
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

In [6]:
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    agents = [agent_1, agent_2, agent_3]
    decisions = []

    pattern = r'"response"\s*:\s*"?([AB])"?'

    ground_truth = "A" if "A>" in label else "B"

    for agent in agents:
            result = initializer.initiate_chat(agent, message=message, max_turns=1)
            result_str = str(result)

            match = re.search(pattern, result_str)
            decision = match.group(1) if match else "X"
            decisions.append(decision)

    if "X" in decisions:
        return {
            "system_decision": "X",
            "ground_truth": ground_truth,
            "is_correct": False
        }

    system_decision = Counter(decisions).most_common(1)[0][0]
    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }


In [7]:
num_rows = 100

df_final_subset = df_final.head(num_rows)

# normal position
results_1 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/parallel_1.csv', index=False)

# swapped position
results_2 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"], # positions swap 
        response_B=row["response_A"], # positions swap 
        label=row["label"],
    )

    # position swap back
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/parallel_2.csv', index=False)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

initializer (to Scientific-Agent):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mol

Progress:   1%|          | 1/100 [00:00<00:19,  5.04it/s]

initializer (to Scientific-Agent):

 
    Question: The total cost of producing x cameras is C(x) = 2 + x^3. What is the average cost if 10 cameras are made? What is the marginal cost of producing 10 cameras?
(A) Average Cost: $1000/unit, Marginal Cost: $100
(B) Average Cost: $300/unit, Marginal Cost: $100
(C) Average Cost: $300/unit, Marginal Cost: $1000
(D) Average Cost: $102/unit, Marginal Cost: $301
(E) Average Cost: $500/unit, Marginal Cost: $1000
(F) Average Cost: $100/unit, Marginal Cost: $100
(G) Average Cost: $200/unit, Marginal Cost: $300
(H) Average Cost: $2/unit, Marginal Cost: $30
(I) Average Cost: $1020/unit, Marginal Cost: $299
(J) Average Cost: $100/unit, Marginal Cost: $300
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1)

Progress:   2%|▏         | 2/100 [00:00<00:17,  5.76it/s]

...

Progress:  98%|█████████▊| 98/100 [15:33<00:17,  8.73s/it]

initializer (to Scientific-Agent):

 
    Question: Two whole numbers have a least common multiple of 60. Each number is less than or equal to 12. The greatest common factor of the two numbersis 2. What are the two numbers?
(A) 5 and 12
(B) 12 and 15
(C) 6 and 12
(D) 4 and 8
(E) 3 and 9
(F) 8 and 10
(G) 6 and 10
(H) 2 and 10
(I) 2 and 6
(J) 10 and 12
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The two numbers have a least common multiple (LCM) of 60.

2) Both numbers are less than or equal to 12.

3) The greatest common factor (GCF) of the two numbers is 2.

4) Let's list the factors of 60: 1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 30, 60

5) From these, we need to find two numbers that are both ≤ 12 and have a GCF of 2.

6) The pairs that f

Progress:  99%|█████████▉| 99/100 [15:43<00:09,  9.07s/it]

initializer (to Scientific-Agent):

 
    Question: Neural networks:
(A) Are incapable of handling non-linear relationships
(B) Can only handle numerical data
(C) None of the above
(D) Can only be trained with stochastic gradient descent
(E) Are only applicable to visual data
(F) Always require large amounts of data to train
(G) Optimize a convex objective function
(H) Can use a mix of different activation functions
(I) Can only use one type of activation function
(J) Cannot be used for classification tasks
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's analyze each option:

(A) False. Neural networks are actually very good at handling non-linear relationships.
(B) False. Neural networks can handle various types of data, including categorical data.
(C) This might be c

Progress: 100%|██████████| 100/100 [15:53<00:00,  9.53s/it]
